<a href="https://colab.research.google.com/github/cokoroafor11/Playlist-Recommender-App/blob/main/Spotify_Playlist_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!pip install spotipy --upgrade
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 10.0 MB/s eta 0:00:00


In [53]:
#Things to call:
#Excel sheet with spotify playlists
playlist_sheet = 'Spotify Playlists 1 per genre.xlsx'

## Create the Spotify object

In [2]:
#Save client credentials
client_id = "c1f74565be774e65aa211462aaf5fed8"
client_secret = "2edce4052f8f46639c0e112658572d66"
#Create object
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,client_secret=client_secret),requests_timeout=100,retries=3)

## Functions I Need

### Extract songs from playlist

In [3]:
def extract_songs(playlist):
    '''Function to extract songs from a playlist: Will run this for all the playlists being added'''

    #Make sure playlist input is string
    if type(playlist) != str: 
        playlist = str(playlist)

    #Get playlist tracks    
    playlist_tracks =  spotify.playlist_tracks(playlist)
    tracks = []

    #Append tracks to list if no type error
    for elem in playlist_tracks['items']:
        try:
            tracks.append(elem['track']['external_urls']['spotify'])
        except TypeError:
            pass
    return tracks
  

In [48]:
tracks = extract_songs('https://open.spotify.com/playlist/37i9dQZEVXbqgPCgwknZJW?si=ae7b2ccfd8d64357')

In [5]:
tracks

['https://open.spotify.com/track/5xP9lQYA8YQmQh6BOxcAnR',
 'https://open.spotify.com/track/1ZsNLqqBzUbDG9LDosVk2p',
 'https://open.spotify.com/track/5viZ8CLipufc1Q1NXpMRNN',
 'https://open.spotify.com/track/3kH94a1MQM7YrpsZRWuXrh',
 'https://open.spotify.com/track/5VUkNfVb58kUWyWoWdbvha',
 'https://open.spotify.com/track/4t48gom0WM1BgWPmC9e8HV',
 'https://open.spotify.com/track/2TtD8MWLffh6SO7YOhELJO',
 'https://open.spotify.com/track/4EZ0ZWTyjIUtZk2Rqf3QAG',
 'https://open.spotify.com/track/2cVIKCgIfYOYyyXfQEstZN',
 'https://open.spotify.com/track/4jeQnCp9IL3pbLPn0UjLmh',
 'https://open.spotify.com/track/3AFxrPHW0222y3t4dmHAJI',
 'https://open.spotify.com/track/6SWOyZfVoLXgqKCKj3hI9c',
 'https://open.spotify.com/track/4WCX5GB4SZ9IDa1ULJNzJk',
 'https://open.spotify.com/track/6UmbweVDCrk7XcC7hicFL7',
 'https://open.spotify.com/track/6LluC6nv9k4h5gwMLjy3FJ',
 'https://open.spotify.com/track/03E7tuKVSY1BS4LaV9rrOf',
 'https://open.spotify.com/track/0KT0GKZJ72959Mk1UE0lE5',
 'https://open

### Get Artists Names

In [6]:
def get_track_artists(track):
    '''Get all artists for each track'''
    
    artists = []
    
    #Iterate through all avaiable artists, append to new array
    for artist in spotify.track(track)['artists']:
        artists.append(artist['name'])
    return artists

In [7]:
get_track_artists('https://open.spotify.com/track/5OSGdSxtulzlzdPdK2MQJC?si=a4a9db4cd27242b2')

['Marshmello', 'Farruko']

### Get Track Name

In [8]:
def get_track_name(track):
    '''Get track name'''

    return spotify.track(track)['name']

### Get Audio Features

In [9]:
def get_audio_features(song, feature):
    '''Function to get score for feature type of a song'''

    feature_score = 0
    #Get audio features if no type error, otherwise skip
    try:
      feature_score = spotify.audio_features(song)[0][feature]
    except TypeError:
      feature_score = None
      pass
    return feature_score

In [10]:
get_audio_features

<function __main__.get_audio_features(song, feature)>

What do I need from each song?
1. Artist names
2. Album
3. Genres
4. Song features


In [11]:
spotify.audio_features("https://open.spotify.com/track/5xP9lQYA8YQmQh6BOxcAnR")[0]

{'danceability': 0.852,
 'energy': 0.675,
 'key': 1,
 'loudness': -6.271,
 'mode': 1,
 'speechiness': 0.197,
 'acousticness': 0.0652,
 'instrumentalness': 3.74e-05,
 'liveness': 0.36,
 'valence': 0.83,
 'tempo': 99.012,
 'type': 'audio_features',
 'id': '5xP9lQYA8YQmQh6BOxcAnR',
 'uri': 'spotify:track:5xP9lQYA8YQmQh6BOxcAnR',
 'track_href': 'https://api.spotify.com/v1/tracks/5xP9lQYA8YQmQh6BOxcAnR',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5xP9lQYA8YQmQh6BOxcAnR',
 'duration_ms': 215467,
 'time_signature': 4}

In [38]:
dic = spotify.audio_features("https://open.spotify.com/track/5xP9lQYA8YQmQh6BOxcAnR")[0]
output = pd.DataFrame()

In [39]:
output = output.append(dic, ignore_index=True)

<ipython-input-39-703685a495c5>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(dic, ignore_index=True)


In [40]:
output = output.iloc[:, 0:11]
output.drop(columns = ['mode', 'key',])

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.852,0.675,-6.271,0.197,0.0652,0.000037,0.36,0.83,99.012


In [41]:
output.insert(loc = 0, column = 'song', value = get_track_name("https://open.spotify.com/track/5xP9lQYA8YQmQh6BOxcAnR"))

In [42]:
output

,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Popular (with Playboi Carti & Madonna),0.852,0.675,1,-6.271,1,0.197,0.0652,0.000037,0.36,0.83,99.012


In [44]:
output.insert(loc = 1, column = 'artists', value = [get_track_artists("https://open.spotify.com/track/5xP9lQYA8YQmQh6BOxcAnR")])

In [45]:
output

,song,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Popular (with Playboi Carti & Madonna),"[The Weeknd, Playboi Carti, Madonna]",0.852,0.675,1,-6.271,1,0.197,0.0652,0.000037,0.36,0.83,99.012


Then I need a function to get songs straight from a playlist, convert to feature info and put into a db

In [14]:
def songs_to_features(song):
  feature_dict = spotify.audio_features(song)
  

In [ ]:
#https://open.spotify.com/playlist/7IKIjt9wZPPQWgvHo1SeFO?si=75371c3c8eaa49a2

In [51]:
songs = extract_songs('https://open.spotify.com/playlist/7IKIjt9wZPPQWgvHo1SeFO?si=75371c3c8eaa49a2')

In [ ]:
#Spotify Playlists 1 per genre.xlsx

In [52]:
def excel_list_to_df():
    '''Create the dataframe from the excel sheet of playlists'''

    df = pd.read_excel("Spotify Playlists 1 per genre.xlsx")
    playlists = df['Link']
    return playlists

Let's have a function that will show stats for the playlist

Have a Map to Show Where 

Going to have the option to either log in and save the playlist or just save it to excel

The flow should probably be as follows:

Part 1: Building the database
1. Extract songs from playlists
2. Perform extraction of all audio features
3. Need a way to append song name and audio feature info to the dataframe, OR perform a function call after the recommended songs are provided to get the song names and artist info

Part 2: Build the recommendations from cosine similarity

Part 3: Integration with Flask or Django